# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123159e+02     1.489365e+00
 * time: 0.7980210781097412
     1     1.116052e+01     9.305279e-01
 * time: 2.1083030700683594
     2    -1.170498e+01     1.040106e+00
 * time: 2.7525479793548584
     3    -3.424346e+01     6.912879e-01
 * time: 3.7094380855560303
     4    -4.788492e+01     4.605837e-01
 * time: 4.630439043045044
     5    -5.710154e+01     1.705422e-01
 * time: 5.563780069351196
     6    -5.994104e+01     9.023609e-02
 * time: 6.208985090255737
     7    -6.094026e+01     5.437975e-02
 * time: 6.819653034210205
     8    -6.133558e+01     6.455901e-02
 * time: 7.4510579109191895
     9    -6.164665e+01     3.946750e-02
 * time: 8.073868036270142
    10    -6.184814e+01     2.818175e-02
 * time: 8.685014009475708
    11    -6.198560e+01     1.799976e-02
 * time: 9.320240020751953
    12    -6.204902e+01     1.789602e-02
 * time: 9.967684984207153
    13    -6.210886e+01     1.797937e-02
 * time: 10.595458030700684
 

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671075
    AtomicLocal         -18.8557662
    AtomicNonlocal      14.8522616
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485385 
    Xc                  -19.3336826

    total               -62.261666458845